In [5]:
import pandas as pd
import os
import time
import requests
import tqdm
import json

In [6]:
ROOT_FOLDER = r"/home/tyra/Documents/CERES/resources"
ROOT_FOLDER = r"C:\Users\Orion\Documents\Projets\CERES\PMA"

### Merge CSVs 

In [ ]:
nb_files = 3
li = []
for i in range(nb_files):
    path = os.path.join(ROOT_FOLDER, f'PMA_clean{i+1}.csv')
    print(path)
    dftemps = pd.read_csv(path, sep=';', index_col=None, header=0)
    li.append(dftemps)
df = pd.concat(li)

In [ ]:
df.to_csv(os.path.join(ROOT_FOLDER, 'PMA_full.csv'), sep=";", index=False)

In [ ]:
df = pd.read_csv(os.path.join(ROOT_FOLDER, 'PMA_full.csv'), sep=";", header=0)

### Count number of tweets for each user

In [ ]:
df.head()

In [ ]:
df.groupby(['from_user_id', 'from_user_name']).size().reset_index(name='counts').sort_values('counts', ascending=False)[:15].to_csv('top_15.csv')

### Count number of mentions 

In [89]:
import re
def count_occurences(df, exp, exp_regex, callback=None):
    def count_mentions(li, res=None, bar=None):
        for el in li:
            key = el
            if callback:
                key = callback(el)
                bar.update(1)
            res[key] = res.get(key, 0) + 1
        
    def find_mentions(value):
        reg = re.compile(exp_regex)
        return reg.findall(value)
    
    contains_mentions = df['text'].str.contains(exp)
    df_mentions = df[contains_mentions]
    
    contains_rt = df_mentions['text'].str.contains(r'RT')
    df_mentions_no_rt = df_mentions[~contains_rt]

    mentions_list = df_mentions['text'].apply(find_mentions)
    mentions_list_no_rt = df_mentions_no_rt['text'].apply(find_mentions)
    
    dic_mentions = {}
    dic_mentions_no_rt = {}
    
    with tqdm.tqdm(total=len(mentions_list)) as pbar:
        mentions_list.apply(count_mentions, res=dic_mentions, bar=pbar)
    with tqdm.tqdm(total=len(mentions_list_no_rt)) as pbar:
        mentions_list_no_rt.apply(count_mentions, res=dic_mentions_no_rt, bar=pbar)
    
    final = pd.DataFrame(columns=['url', 'count'])
    final['url'] = dic_mentions.keys()
    final['count'] = dic_mentions.values()
    
    final_no_rt = pd.DataFrame(columns=['url', 'count'])
    final_no_rt['url'] = dic_mentions_no_rt.keys()
    final_no_rt['count'] = dic_mentions_no_rt.values()
    
    final = final.merge(final_no_rt, on='url')
    final = final.rename(columns={'count_x': 'total_count', 'count_y': 'count_without_RT'})
    final['pourcentage_vraies_mentions'] = round(final['count_without_RT'] / final['total_count'] * 100, 1)
    final = final.sort_values('total_count', ascending=False)
    
    return final

In [ ]:
def find_mentions(value):
        reg = re.compile(regex)
        return reg.findall(value)

In [ ]:
contains_mentions = df['text'].str.contains(r'https://')

In [ ]:
len(contains_mentions)

In [ ]:
df_mentions = df[contains_mentions]

In [ ]:
mentions_list = df_mentions['text'].apply(find_mentions)

In [ ]:
mentions_list[0]

In [8]:
with open(os.path.join(ROOT_FOLDER, 'urls.json'), 'r') as f:
    urls = json.load(f)

In [51]:
def get_source(url):
    if url not in urls:
        return url
    return urls[url]

In [90]:
regex = "(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})"
res = count_occurences(df, r'https?://', regex, callback=get_source)
res.head()

1427103it [00:04, 329738.05it/s]                                                                                       
168863it [00:00, 298880.73it/s]                                                                                        


,url,total_count,count_without_RT,pourcentage_vraies_mentions
0,twitter.com,998196,75740,7.6
5,www.lefigaro.fr,24674,3596,14.6
33,t.co,22650,4409,19.5
52,www.youtube.com,19060,5458,28.6
13,www.valeursactuelles.com,17347,1455,8.4


In [98]:
res.drop(res[res.url == 't.co'].index, inplace=True)
res.head()

,url,total_count,count_without_RT,pourcentage_vraies_mentions
5,www.lefigaro.fr,24674,3596,14.6
52,www.youtube.com,19060,5458,28.6
13,www.valeursactuelles.com,17347,1455,8.4
64,www.lemonde.fr,14337,2943,20.5
6,www.francetvinfo.fr,13288,2687,20.2


In [99]:
res = res.rename(columns={'url': 'source', 'pourcentage_vraies_mentions': 'pourcentage_hors_rt'})
res

,source,total_count,count_without_RT,pourcentage_hors_rt
5,www.lefigaro.fr,24674,3596,14.6
52,www.youtube.com,19060,5458,28.6
13,www.valeursactuelles.com,17347,1455,8.4
64,www.lemonde.fr,14337,2943,20.5
6,www.francetvinfo.fr,13288,2687,20.2
...,...,...,...,...
3802,modalis.fr,1,1,100.0
3804,hedahere.wordpress.com,1,1,100.0
3805,modern.hypnoweb.net,1,1,100.0
3806,www.merlanfrit.net,1,1,100.0


In [100]:
res.to_csv(os.path.join(ROOT_FOLDER, 'urls.csv'), index=False, sep=";")

### Expand all urls

In [9]:
regex = "(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})"
metions = df['text'].str.extractall(regex)

In [10]:
uniques_url = metions.dropna().drop_duplicates()

In [23]:
'https://t.co/gJPQDDXR8t' in urls

False

In [11]:
urls_values = uniques_url[0].values

In [30]:
len(urls.keys())

180501

In [50]:
remaining = list(set(urls_values) - set(urls.keys()))
len(remaining)

113

In [48]:
def get_source(url, bar=None):
    if url in urls:
        bar.update(1)
        return
    try:
        if not url.startswith('http'):
            url = 'http://' + url
        res = requests.get(url, timeout=0.5)
        full_url = res.url
    except Exception as err:
        print(err.request.url)
        full_url = err.request.url
    source = full_url.split('//')[1].split('/')[0]
    urls[url] = source
    bar.update(1)
    return source

In [49]:
import concurrent

with concurrent.futures.ThreadPoolExecutor() as executor:
    with tqdm.tqdm(total=len(remaining)) as pbar:
        res = [executor.submit(get_source, url, pbar) for url in remaining]
        concurrent.futures.wait(res)

  2%|█▍                                                                                | 2/114 [00:01<01:41,  1.11it/s]

https://www.valeursactuelles.com/clubvaleurs/societe/en-attendant-taubira/


  4%|███▌                                                                              | 5/114 [00:02<00:38,  2.82it/s]

https://www.infogm.org/6942-bioethique-animaux-chimeres-producteurs-organes-humains


 13%|██████████▋                                                                      | 15/114 [00:06<00:49,  1.99it/s]

...

 17%|█████████████▌                                                                   | 19/114 [00:08<00:41,  2.29it/s]

https://www.breizh-info.com/2021/01/29/158092/marchons-enfants-contre-la-pma-sans-pere-et-la-gpa-des-manifestations-ce-week-end-en-bretagne/


 18%|██████████████▏                                                                  | 20/114 [00:09<00:54,  1.72it/s]

https://www.ludolegars.fr/09-01-2021-expedition-a-newdale-chapitre-3-furnace/


 18%|██████████████▏                                                                  | 20/114 [00:31<02:29,  1.59s/it]


In [17]:
urls

{'https://t.co/TDnZJy7GpP': 'twitter.com',
 'https://t.co/f6u8eIan6I': 'twitter.com',
 'https://t.co/MTkOELwqas': 'twitter.com',
 'https://t.co/fuWswqWbSW': 'twitter.com',
 'https://t.co/K8Hj2DSN88': 'twitter.com',
 'https://t.co/Ob45LHI0Vo': 'twitter.com',
 'https://t.co/mQX6oXD5zU': 'w.lpnt.fr',
 'https://t.co/vQS6w9M6lE': 'twitter.com',
 'https://t.co/HjKtUiXYOe': 'twitter.com',
 'https://t.co/P0BVcREXwF': 'twitter.com',
 'https://t.co/XkPW3Dax1w': 'twitter.com',
 'https://t.co/6KDvtHDajA': 'twitter.com',
 'https://t.co/IWwVM60jZ6': 'twitter.com',
 'https://t.co/XjE8pMgoX6': 'twitter.com',
 'https://t.co/RnvjqgLlU0': 'twitter.com',
 'https://t.co/WfjxuuqP2Z': 'twitter.com',
 'https://t.co/Qa4bDnGHCf': 'twitter.com',
 'https://t.co/eyv37A3nx6': 'twitter.com',
 'https://t.co/0ppGt6dgy1': 'www.lexpress.fr',
 'https://t.co/NxeL5EVUDd': 'twitter.com',
 'https://t.co/Lur1TjbTL2': 'www.europe1.fr',
 'https://t.co/fciO95p4Le': 'twitter.com',
 'https://t.co/ylepMnOtIB': 'twitter.com',
 'http

In [36]:
with open(os.path.join(ROOT_FOLDER, 'urls.json'), 'w') as f:
    json.dump(urls, f)

In [ ]:
df_mentions = df[contains_mentions]

In [ ]:
urls

In [ ]:
contains_rt = df_mentions['text'].str.contains(r'RT')

In [ ]:
df_mentions_no_rt = df_mentions[~contains_rt]

In [ ]:
import re
def find_mentions(value):
    reg = re.compile(r"@\w+")
    return reg.findall(value)

In [ ]:
dic = {}

In [ ]:
def count_mentions(li):
    for el in li:
        dic[el] = dic.get(el, 0) + 1

In [ ]:
mentions_list = df_mentions['text'].apply(find_mentions)

In [ ]:
mentions_list.apply(count_mentions)

In [ ]:
final = pd.DataFrame(columns=['username', 'count'])
final['username'] = dic.keys()
final['count'] = dic.values()

In [ ]:
df2 = pd.read_csv(os.path.join(ROOT_FOLDER, 'mentions.csv'), sep=";")

In [ ]:
len(df2["count"])

In [ ]:
len(final['count'])

In [ ]:
final = final.merge(df2, on='username')

In [ ]:
final = final.rename(columns={'count_x': 'count_with_RT', 'count_y': 'count_without_RT'})

In [ ]:
final['pourcentage_vraies_mentions'] = round(final['count_without_RT'] / final['count_with_RT'] * 100, 1)

In [ ]:
final.sort_values('count_with_RT', ascending=False).to_csv(os.path.join(ROOT_FOLDER, 'mentions.csv'), index=False, sep=";")

In [ ]:
t = s.split('\n')
ss = t[305: 307]